In [1]:
import os
os.chdir("../")

In [2]:
from src.utils.seed_everything import seed_everything

seed_everything()

In [3]:
import torch

In [4]:
BATCH_SIZE = 8
DIMS = 128
NEGS = 4

queries = torch.randn(BATCH_SIZE, DIMS)
docs = torch.randn(BATCH_SIZE, NEGS, DIMS)
labels = torch.randint(0, NEGS, (BATCH_SIZE,))


## Using cosine similarity

In [5]:
import torch.nn.functional as F

In [6]:
similarities = F.cosine_similarity(queries.unsqueeze(1), docs, dim=2)
print(similarities)


tensor([[ 0.0056,  0.0078, -0.0440, -0.0220],
        [-0.1620, -0.1334, -0.0496, -0.0564],
        [ 0.2566,  0.1775, -0.0062,  0.1534],
        [ 0.0719, -0.0746,  0.0604,  0.0637],
        [-0.0932,  0.0905,  0.0314,  0.0192],
        [-0.0003,  0.0033,  0.0195, -0.0713],
        [ 0.1325, -0.0115, -0.0031,  0.0493],
        [ 0.1178,  0.1071,  0.0044, -0.0481]])


## Manual

In [7]:
similarities_2 = torch.sum(
    F.normalize(queries.unsqueeze(1)) * F.normalize(docs),
    dim=2,
)
print(similarities_2.shape)


torch.Size([8, 4])


In [8]:
torch.allclose(similarities, similarities_2)

False

In [9]:
similarities_2

tensor([[ -3.4410,  -2.0448,  -3.8589,  -8.3760],
        [ -4.7184,  -4.8038,   0.4652,  -0.8647],
        [  6.3148,   7.9450,  -5.4284,   8.0711],
        [  3.7961, -10.2951,   1.6157,   1.8545],
        [ -4.8166,   7.5033,  -1.8139,  -1.9209],
        [ -3.5051,  -4.1554,   1.8612,  -1.0211],
        [ 13.7368,  -2.2949,   3.4392,   5.5150],
        [  5.0538,   6.5248,   2.0178,   1.5827]])

In [10]:
similarities

tensor([[ 0.0056,  0.0078, -0.0440, -0.0220],
        [-0.1620, -0.1334, -0.0496, -0.0564],
        [ 0.2566,  0.1775, -0.0062,  0.1534],
        [ 0.0719, -0.0746,  0.0604,  0.0637],
        [-0.0932,  0.0905,  0.0314,  0.0192],
        [-0.0003,  0.0033,  0.0195, -0.0713],
        [ 0.1325, -0.0115, -0.0031,  0.0493],
        [ 0.1178,  0.1071,  0.0044, -0.0481]])

## Loss Calculation

In [11]:
mask = torch.ones_like(similarities)
mask[torch.arange(BATCH_SIZE), labels] = 0

In [12]:
temperature = 0.07
exp = torch.exp(similarities / temperature)

In [13]:
pos_exp = exp[mask == 0].view(BATCH_SIZE, 1)

In [14]:
loss = -torch.log(pos_exp / exp.sum(dim=1))
loss.mean()


tensor(1.2697)